In [2]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns

# Lab | Inferential statistics - T-test & P-value

### Instructions

1. *One tailed t-test* - In a packing plant, a machine packs cartons with jars. It is supposed that a new machine will pack faster on the average than the machine currently used. To test that hypothesis, the times it takes each machine to pack ten cartons are recorded. The results, in seconds, are shown in the tables in the file `files_for_lab/machine.txt`.
   Assume that there is sufficient evidence to conduct the t test, does the data provide sufficient evidence to show if one machine is better than the other?



In [37]:
new_machine = [42.1, 41, 41.3, 41.8, 42.4, 42.8, 43.2, 42.3, 41.8, 42.7]

In [38]:
old_machine = [42.7, 43.6, 43.8, 43.3, 42.5, 43.5, 43.1, 41.7, 44, 44.1]   

In [40]:
dic = {'old_machine': old_machine, 'new_machine': new_machine} 
dic

{'old_machine': [42.7, 43.6, 43.8, 43.3, 42.5, 43.5, 43.1, 41.7, 44, 44.1],
 'new_machine': [42.1, 41, 41.3, 41.8, 42.4, 42.8, 43.2, 42.3, 41.8, 42.7]}

In [41]:
df = pd.DataFrame(dic)
df 

old_machine  new_machine
0         42.7         42.1
1         43.6         41.0
2         43.8         41.3
3         43.3         41.8
4         42.5         42.4
5         43.5         42.8
6         43.1         43.2
7         41.7         42.3
8         44.0         41.8
9         44.1         42.7

We conduct a one-side t test with pooled samples as the two samples are independent, we cannot match the observations to one another.

H0: the time it takes to the old and new machines to pack 10 jars is the same. The mean of the new_machine is inferior or equal to the mean of the old_machine: sample_new_mean<=sample_old_mean.

H1: the time it takes to pack 10 jars is higher for the new machine:  mean(new_machine)> mean(old_machine). Right side test.
Significance level: 0.05, confidence interval 95%. 
We compute the t statistic:

In [56]:
sample_new_mean, sample_old_mean = df['new_machine'].mean(), df['old_machine'].mean()
sample_new_std, sample_old_std = df['new_machine'].std(ddof=1), df['old_machine'].std(ddof=1)
sp = ( len(df) - 1 ) * ( sample_new_std**2 ) +  ( len(df) - 1 ) * ( sample_old_std**2 )
sp /= ( len(df) + len(df) - 2) #sp = sp/
r = np.sqrt( (1/len(df)) + (1/len(df)) )
t = (sample_new_mean - sample_old_mean )/ (sp * r)

print("The mean of sample new_machine is {:.2f}".format(sample_new_mean))
print("The mean of sample old_machine is {:.2f}".format(sample_old_mean))
print("The standard deviation of sample new_machine is {:.2f}".format(sample_new_std))
print("The standard deviation of sample old_machine is {:.2f}".format(sample_old_std))
print("The ratio of the sample variances is {:.2f} which is bigger than 0.5 and smaller than 2".format(sample_new_std/sample_old_std))
print("The t statistic is: {:.2f}".format(t))

The mean of sample new_machine is 42.14
The mean of sample old_machine is 43.23
The standard deviation of sample new_machine is 0.68
The standard deviation of sample old_machine is 0.75
The ratio of the sample variances is 0.91 which is bigger than 0.5 and smaller than 2
The t statistic is: -4.74


Test with critical values: 

In [57]:
st.t.ppf((1-0.05), df = len(df)+len(df)-2) 

1.7340636066175354

Our t statistic is -4.74 and the critical value is for a significance level of 0.05 is 1.73. We accept the H0, the new machine is faster than the old machine. 

Test with p_values:

In [53]:
st.t.cdf(t,df = len(df)+len(df)-2) ###Ya no lo entiendo. 

8.260617649942772e-05

2. *Matched Pairs Test* - In this challenge we will compare dependent samples of data describing our Pokemon (file `files_for_lab/pokemon.csv`). Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. Compare the two columns to see if there is a statistically significant difference between them and comment your result.

In [60]:
df1= pd.read_csv('pokemon.csv')
df1

#                   Name   Type 1  Type 2  Total  HP  Attack  Defense  \
0      1              Bulbasaur    Grass  Poison    318  45      49       49   
1      2                Ivysaur    Grass  Poison    405  60      62       63   
2      3               Venusaur    Grass  Poison    525  80      82       83   
3      3  VenusaurMega Venusaur    Grass  Poison    625  80     100      123   
4      4             Charmander     Fire     NaN    309  39      52       43   
..   ...                    ...      ...     ...    ...  ..     ...      ...   
795  719                Diancie     Rock   Fairy    600  50     100      150   
796  719    DiancieMega Diancie     Rock   Fairy    700  50     160      110   
797  720    HoopaHoopa Confined  Psychic   Ghost    600  80     110       60   
798  720     HoopaHoopa Unbound  Psychic    Dark    680  80     160       60   
799  721              Volcanion     Fire   Water    600  80     110      120   

     Sp. Atk  Sp. Def  Speed  Generation  Legendary  
0         65       65     45           1      False  
1         80       80     60           1      False  
2        100      100     80           1      False  
3        122      120     80           1      False  
4         60       50     65           1      False  
..       ...      ...    ...         ...        ...  
795      100      150     50           6       True  
796      160      110    110           6       True  
797      150      130     70           6       True  
798      170      130     80           6       True  
799      130       90     70           6       True  

[800 rows x 13 columns]

We want to know if there is a significant difference between the attack and defense features of the pokemon:
We fix significance level to 0.05.
H0: there is no significant difference and hence the difference of the means is equal to zero.
H1: there is a significant difference and hence the difference of the means is different than zero. Two-sided test. 

In [61]:
sample = df1.sample(30, random_state=1)
sample['difference'] = sample['Attack']-sample['Defense']
sample.head()

#                       Name  Type 1  Type 2  Total  HP  Attack  \
8      6  CharizardMega Charizard Y    Fire  Flying    634  78     104   
510  460                  Abomasnow   Grass     Ice    494  90      92   
175  161                    Sentret  Normal     NaN    215  35      46   
735  667                     Litleo    Fire  Normal    369  62      50   
242  224                  Octillery   Water     NaN    480  75     105   

     Defense  Sp. Atk  Sp. Def  Speed  Generation  Legendary  difference  
8         78      159      115    100           1      False          26  
510       75       92       85     60           4      False          17  
175       34       35       45     20           2      False          12  
735       58       73       54     72           6      False          -8  
242       75      105       75     45           2      False          30

We compute the t statistic: 

In [62]:
sample_diff_mean, sample_diff_std = sample['difference'].mean(), sample['difference'].std(ddof=1)
sample_diff_mean, sample_diff_std

(2.8, 36.32079294288603)

In [63]:
t = sample_diff_mean / ( sample_diff_std / np.sqrt(sample.shape[0]) )
print("The mean of our samples differences is: {:.2f}".format(sample_diff_mean))
print("The standard deviation of our samples differences is: {:.2f}".format(sample_diff_std))
print("Our t statistics is: {:.2f}".format(t))

The mean of our samples differences is: 2.80
The standard deviation of our samples differences is: 36.32
Our t statistics is: 0.42


In [64]:
tc = st.t.ppf(1-(0.05/2),df= sample.shape[0] - 1)
tc

2.045229642132703


# OPTIONAL PART | Inferential statistics - ANOVA

Note: The following lab is divided in 2 sections which represent activities 3 and 4.

## Part 1

In this activity, we will look at another example. Your task is to understand the problem and write down all the steps to set up ANOVA. After the next lesson, we will ask you to solve this problem using Python. Here are the steps that you would need to work on:
    - Null hypothesis
    - Alternate hypothesis
    - Level of significance
    - Test statistic
    - P-value
    - F table

### Context

Suppose you are working as an analyst in a microprocessor chip manufacturing plant. You have been given the task of analyzing a plasma etching process with respect to changing Power (in Watts) of the plasma beam. Data was collected and provided to you to conduct statistical analysis and check if changing the power of the plasma beam has any effect on the etching rate by the machine. You will conduct ANOVA and check if there is any difference in the mean etching rate for different levels of power. You can find the data `anova_lab_data.xlsx` file in the `files_for_lab` folder  

- State the null hypothesis
- State the alternate hypothesis
- What is the significance level
- What are the degrees of freedom of model, error terms, and total DoF

Data was collected randomly and provided to you in the table as shown: [link to the image - Data](https://education-team-2020.s3-eu-west-1.amazonaws.com/data-analytics/7.05/7.05-lab_data.png)


## Part 2

- In this section, use the Python to conduct ANOVA.
- What conclusions can you draw from the experiment and why?


In [3]:
df2= pd.read_excel('anova_lab_data.xlsx')
df2

Power   Etching Rate
0   160 W          5.43
1   180 W          6.24
2   200 W          8.79
3   160 W          5.71
4   180 W          6.71
5   200 W          9.20
6   160 W          6.22
7   180 W          5.98
8   200 W          7.90
9   160 W          6.01
10  180 W          5.66
11  200 W          8.15
12  160 W          5.59
13  180 W          6.60
14  200 W          7.55

In [4]:
def stHead(df):
    new_header = []
    for h in df.columns:
        x = h.lower().replace(' ','')
        new_header.append(x)
    df.columns = new_header

In [5]:
stHead(df2)

In [6]:
df2.head()

power  etchingrate
0  160 W         5.43
1  180 W         6.24
2  200 W         8.79
3  160 W         5.71
4  180 W         6.71

In [7]:
df2['power'].value_counts()

160 W    5
180 W    5
200 W    5
Name: power, dtype: int64

We have three (K) power groups, with a number of observations per group = 5.

## Part 1. Setting the hypothesis

H0: the means of the three possible values for power are equal. 
H1: the means are different.

We fix a significance level of 0.05. 

In [9]:
group_df2 = df2.groupby('power')['etchingrate'].agg(power_mean='mean',Samples='size').reset_index()
group_df2

power  power_mean  Samples
0  160 W       5.792        5
1  180 W       6.238        5
2  200 W       8.318        5

In [ ]:
The degrees of freedom of model are: 
The error terms, and total DoF

## Part 2. Conducting the ANOVA